
# AR → Yale 비지도 학습 챌린지 (**우승 시 A+**)

이 노트북은 운영진이 제공한 **비식별화 데이터셋(`dist_students/`)** 과 **학습용 AR 데이터(npz)** 를 사용해
- **완전 비지도 학습**(라벨 미사용)으로 표현을 학습하고,
- **Public‑Val**에서 성능 지표(**Hungarian ACC / NMI / ARI**)를 계산해 모델을 개선하며,
- 최종 제출파일 **`submission.csv`**(헤더: `id,pred`)을 생성·검증하는 과제입니다.

> **규칙 요약**  
> • 학습 단계에서 라벨(정답) **사용 금지**.  
> • Public‑Val의 라벨은 **평가/튜닝 판단** 용도로만 사용(모델 학습/미세조정에 사용 금지).  
> • 최종 점수는 비공개 **Private‑Test** 라벨로 출제진이 채점합니다.


## 1) 환경 준비


In [12]:

# 필요 시 주석 해제하여 설치하세요.
# !pip install numpy pandas pillow scikit-learn tqdm matplotlib

import os, json, glob
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image, ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import normalized_mutual_info_score as NMI
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from scipy.optimize import linear_sum_assignment
from skimage.feature import hog, local_binary_pattern
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import Normalizer


import matplotlib.pyplot as plt

## 2) 데이터 로딩 (배포된 인덱스 사용)


In [13]:

# 이 경로대로 경로를 맞추세요.
PUB_CSV = "dist_students/public_val_index.csv"     # id, filename, true_label
PRV_CSV = "dist_students/private_test_index.csv"   # id, filename
TRAIN_NPZ = "data_ar/ar_database_stacked_128x128.npz"  # 라벨 없는 AR 데이터

# 인덱스 로딩
pub = pd.read_csv(PUB_CSV)
prv = pd.read_csv(PRV_CSV)

print("Public‑Val:", pub.shape, " | Private‑Test:", prv.shape)
display(pub.head(3))
display(prv.head(3))

prv_df = pd.read_csv("dist_students/private_test_index.csv")
pub_df = pd.read_csv("dist_students/public_val_index.csv")


Public‑Val: (45, 3)  | Private‑Test: (120, 2)


,id,filename,true_label
0,b13c4cc493,dist_students/all/b13c4cc493.png,0
1,5fd5d507da,dist_students/all/5fd5d507da.png,0
2,a409d091eb,dist_students/all/a409d091eb.png,0


,id,filename
0,4ba8a5797a,dist_students/all/4ba8a5797a.png
1,4e4063e835,dist_students/all/4e4063e835.png
2,759073b6ce,dist_students/all/759073b6ce.png


## 3) 학습 데이터(AR .npz) 로딩 및 전처리 ((3315, 128, 128))


In [28]:
if os.path.exists(PUB_CSV) and os.path.exists(PRV_CSV):
    pub = pd.read_csv(PUB_CSV)
    prv = pd.read_csv(PRV_CSV)

def apply_gamma_correction(img, gamma=0.5):   # 감마보정
    img_arr = np.array(img, dtype=np.float32) / 255.0
    img_gamma = np.power(img_arr, gamma)
    img_gamma = (img_gamma * 255).astype(np.uint8)
    return Image.fromarray(img_gamma)

def extract_lbp_features(img_arr_2d):   # LBP(질감 추출)
    radius = 2
    n_points = 8 * radius
    lbp = local_binary_pattern(img_arr_2d, n_points, radius, method='uniform')

    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
    return hist

def extract_hog_features(img_arr_2d):   # HOG(얼굴 구조 특징 추출)
    return hog(img_arr_2d, orientations=9, pixels_per_cell=(8, 8),
               cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)

def load_ar_data_ultimate(npz_path):    # AR 데이터 전처리
    if not os.path.exists(npz_path): return None
    data = np.load(npz_path)
    key = list(data.keys())[0]
    ar_imgs = data[key]

    fusion_list = []
    print(f"[AR Data] Extracting Ultimate Features...")
    for img_arr in ar_imgs:
        img = Image.fromarray(img_arr)

        img_gamma = apply_gamma_correction(img, gamma=0.4)
        img_gamma_arr = np.array(img_gamma)

        hog_vec = extract_hog_features(img_gamma_arr)

        lbp_vec = extract_lbp_features(img_gamma_arr)

        raw_vec = np.array(img_gamma.resize((32, 32))).flatten() * 0.5

        fusion_vec = np.concatenate([hog_vec, lbp_vec, raw_vec])
        fusion_list.append(fusion_vec)

    return np.array(fusion_list, dtype=np.float32)

def load_yale_images_ultimate(df):    #Yale 이미지 전처리
    fusion_list = []
    for idx, row in df.iterrows():
        path = row['filename']
        if not os.path.exists(path): continue

        img = Image.open(path).convert('L').resize((128, 128))

        img_gamma = apply_gamma_correction(img, gamma=0.4)
        img_gamma_arr = np.array(img_gamma)

        hog_vec = extract_hog_features(img_gamma_arr)

        lbp_vec = extract_lbp_features(img_gamma_arr)

        raw_vec = np.array(img_gamma.resize((32, 32))).flatten() * 0.5

        fusion_vec = np.concatenate([hog_vec, lbp_vec, raw_vec])
        fusion_list.append(fusion_vec)

    return np.array(fusion_list, dtype=np.float32)

print(">>> Loading Data & Extracting ULTIMATE Features...")
X_ar = load_ar_data_ultimate(TRAIN_NPZ)
X_pub = load_yale_images_ultimate(pub)
X_prv = load_yale_images_ultimate(prv)

print(f"Ultimate Feature Shape: {X_ar.shape}")

scaler = StandardScaler()
X_ar_scaled = scaler.fit_transform(X_ar)
X_pub_scaled = scaler.transform(X_pub)
X_prv_scaled = scaler.transform(X_prv)

print(">>> Preprocessing Done.")

>>> Loading Data & Extracting ULTIMATE Features...
[AR Data] Extracting Ultimate Features...
Ultimate Feature Shape: (3315, 9142)
>>> Preprocessing Done.



## 4) 비지도 학습 모델 만들기


In [23]:
def hungarian_acc(y_true, y_pred):

    y_true_int = pd.factorize(y_true)[0].astype(int)
    y_pred_int = np.asarray(y_pred, dtype=int)

    D = max(y_true_int.max(), y_pred_int.max()) + 1
    W = np.zeros((D, D), dtype=np.int64)
    for t, p in zip(y_true_int, y_pred_int):
        W[p, t] += 1

    r, c = linear_sum_assignment(W.max() - W)
    matched = sum(W[ri, ci] for ri, ci in zip(r, c))
    return matched / len(y_true_int)

K_CLUSTERS = len(pub["true_label"].unique())  # 사람 수 = 15
N_COMP_CANDIDATES = [0.90, 0.95, 0.98]
N_NEIGHBORS_CANDIDATES = [5, 8, 10, 12, 15]
AFFINITY = "nearest_neighbors"

normalizer = Normalizer(norm="l2")

best_score = -1.0
best_params = None
best_labels_all = None
best_Z_pub = None
best_Z_prv = None

for n_comp in N_COMP_CANDIDATES:
    pca = PCA(n_components=n_comp, whiten=True, random_state=42)
    pca.fit(X_ar_scaled)

    Z_pub_pca = pca.transform(X_pub_scaled)
    Z_prv_pca = pca.transform(X_prv_scaled)

    Z_pub_norm = normalizer.fit_transform(Z_pub_pca)
    Z_prv_norm = normalizer.transform(Z_prv_pca)
    Z_all = np.vstack([Z_pub_norm, Z_prv_norm])

    for nn in N_NEIGHBORS_CANDIDATES:
        try:
            sp = SpectralClustering(
                n_clusters=K_CLUSTERS,
                affinity=AFFINITY,
                n_neighbors=nn,
                assign_labels="discretize",
                random_state=42,
                n_jobs=-1,
            )

            labels_all = sp.fit_predict(Z_all)
            labels_pub = labels_all[: len(Z_pub_norm)]

            acc = hungarian_acc(pub["true_label"].to_numpy(), labels_pub)
            print(f"   [PCA={n_comp}, nn={nn}] Hungarian ACC (Public) = {acc:.6f}")

            if acc > best_score:
                best_score = acc
                best_params = {"pca": n_comp, "n_neighbors": nn}
                best_labels_all = labels_all
                best_Z_pub = Z_pub_norm
                best_Z_prv = Z_prv_norm
                print(f"   [NEW BEST] ACC={acc:.6f}, params={best_params}")
        except Exception as e:
            print(f"   [Skip] PCA={n_comp}, nn={nn}, error={e}")
            continue

print(f"BEST PARAMS: {best_params}, BEST Hungarian ACC (Public) = {best_score:.6f}")

Z_pub = best_Z_pub
Z_prv = best_Z_prv

class SpectralModelWrapper:
    def __init__(self, Z_pub_ref, Z_prv_ref, labels_all):
        self.Z_pub_ref = np.asarray(Z_pub_ref)
        self.Z_prv_ref = np.asarray(Z_prv_ref)
        self.labels_all = np.asarray(labels_all, dtype=int)
        self.n_pub = self.Z_pub_ref.shape[0]
        self.n_prv = self.Z_prv_ref.shape[0]

    def predict(self, Z):
        Z = np.asarray(Z)
        if Z.shape[0] == self.n_pub:
            return self.labels_all[: self.n_pub]
        elif Z.shape[0] == self.n_prv:
            return self.labels_all[self.n_pub : self.n_pub + self.n_prv]
        else:
            raise ValueError(
                f"Unexpected input shape {Z.shape}, "
                f"expected (n_pub={self.n_pub}, ...) or (n_prv={self.n_prv}, ...)"
            )

model = SpectralModelWrapper(Z_pub, Z_prv, best_labels_all)

pred_pub = model.predict(Z_pub) # Public-Val 예측 결과 (모델 출력)
pred_prv = model.predict(Z_prv) # Private-Test 예측 결과 (모델 출력)

/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


   [PCA=0.9, nn=5] Hungarian ACC (Public) = 0.755556
   [NEW BEST] ACC=0.755556, params={'pca': 0.9, 'n_neighbors': 5}
   [PCA=0.9, nn=8] Hungarian ACC (Public) = 0.733333
   [PCA=0.9, nn=10] Hungarian ACC (Public) = 0.666667
   [PCA=0.9, nn=12] Hungarian ACC (Public) = 0.688889
   [PCA=0.9, nn=15] Hungarian ACC (Public) = 0.644444


/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


   [PCA=0.95, nn=5] Hungarian ACC (Public) = 0.733333
   [PCA=0.95, nn=8] Hungarian ACC (Public) = 0.755556
   [PCA=0.95, nn=10] Hungarian ACC (Public) = 0.711111
   [PCA=0.95, nn=12] Hungarian ACC (Public) = 0.666667
   [PCA=0.95, nn=15] Hungarian ACC (Public) = 0.688889
   [PCA=0.98, nn=5] Hungarian ACC (Public) = 0.711111
   [PCA=0.98, nn=8] Hungarian ACC (Public) = 0.755556
   [PCA=0.98, nn=10] Hungarian ACC (Public) = 0.688889
   [PCA=0.98, nn=12] Hungarian ACC (Public) = 0.688889
   [PCA=0.98, nn=15] Hungarian ACC (Public) = 0.644444
BEST PARAMS: {'pca': 0.9, 'n_neighbors': 5}, BEST Hungarian ACC (Public) = 0.755556


/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


## 5) 제출 파일 생성 및 형식 검증 (`submission.csv`)


In [24]:
# submission.csv는 id와 prediction 값이 2열로 포함되어야 합니다.

def build_submission(pub_df, prv_df, pred_pub, pred_prv, out_csv="submission.csv"):
    sub_pub = pd.DataFrame({"id": pub_df["id"].astype(str), "pred": pd.Series(pred_pub).astype(int)})
    sub_prv = pd.DataFrame({"id": prv_df["id"].astype(str), "pred": pd.Series(pred_prv).astype(int)})
    sub = pd.concat([sub_pub, sub_prv], ignore_index=True)
    sub.to_csv(out_csv, index=False, encoding="utf-8")
    print(f"[Saved] {out_csv} (rows={len(sub)})")
    return out_csv

def validate_submission(sub_csv="submission.csv", pub_csv=PUB_CSV, prv_csv=PRV_CSV):
    sub = pd.read_csv(sub_csv)
    pub = pd.read_csv(pub_csv)
    prv = pd.read_csv(prv_csv)
    # 헤더
    assert list(sub.columns) == ["id","pred"], "헤더는 반드시 'id,pred' 이어야 합니다."
    # ID 집합 확인
    all_ids = set(pub["id"].astype(str)) | set(prv["id"].astype(str))
    assert len(sub) == len(all_ids), f"행 수 불일치: 예상 {len(all_ids)}행, 실제 {len(sub)}행"
    assert sub["id"].is_unique, "id 중복이 있습니다."
    assert set(sub["id"].astype(str)) == all_ids, "제출 id 집합이 인덱스와 일치해야 합니다."
    # pred 정수형 확인
    pd.to_numeric(sub["pred"], errors="raise")
    print("[OK] 제출 파일 형식 검증 통과")
    return True

_ = build_submission(pub, prv, pred_pub, pred_prv, "submission.csv")
validate_submission("submission.csv")


[Saved] submission.csv (rows=165)
[OK] 제출 파일 형식 검증 통과


True

## 6) Public‑Val 성능 평가 (Hungarian ACC / NMI / ARI)


In [25]:

def _to_compact_int_labels(arr: np.ndarray):
    s = pd.Series(arr)
    v = pd.to_numeric(s, errors="raise").to_numpy()
    if np.any(np.isnan(v)):
        raise ValueError("라벨에 숫자가 아닌 값이 포함되어 있습니다.")
    v = v.astype(int)
    uniq = np.unique(v)
    to_compact = {val:i for i,val in enumerate(uniq)}
    inv = {i:val for val,i in to_compact.items()}
    comp = np.array([to_compact[val] for val in v], dtype=int)
    return comp, to_compact, inv

def robust_hungarian_acc(y_true_raw, y_pred_raw):
    y_true, tmap, tinv = _to_compact_int_labels(y_true_raw)
    y_pred, pmap, pinv = _to_compact_int_labels(y_pred_raw)
    Kt, Kp = len(tmap), len(pmap)
    W = np.zeros((Kp, Kt), dtype=np.int64)   # 행=pred, 열=true
    for t, p in zip(y_true, y_pred):
        W[p, t] += 1
    if W.size == 0:
        return 0.0, W, {}
    cost = W.max() - W
    r, c = linear_sum_assignment(cost)
    matched = int(sum(W[int(ri), int(ci)] for ri, ci in zip(r, c)))
    acc = matched / len(y_true) if len(y_true) > 0 else 0.0
    mapping = {list(pmap.keys())[int(ri)]: list(tmap.keys())[int(ci)] for ri, ci in zip(r, c)}
    return acc, W, mapping

merged = pub.merge(pd.read_csv("submission.csv"), on="id", how="inner")
y_true = merged["true_label"].to_numpy()
y_pred = merged["pred"].to_numpy()

acc, W, mapping = robust_hungarian_acc(y_true, y_pred)
nmi = float(NMI(y_true, y_pred))
ari = float(ARI(y_true, y_pred))

print(f"Hungarian ACC: {acc:.6f} | NMI: {nmi:.6f} | ARI: {ari:.6f}")


Hungarian ACC: 0.755556 | NMI: 0.858746 | ARI: 0.590955


## 7) (선택) 라벨 없이 확인할 수 있는 클러스터 품질 지표


In [26]:

def evaluate_unsupervised_quality(embeddings, pred_labels):
    # pred_labels에 클러스터가 2개 미만이면 일부 지표가 정의되지 않습니다.
    if len(np.unique(pred_labels)) < 2:
        return {"silhouette": 0.0, "calinski_harabasz": 0.0, "davies_bouldin": float("inf")}
    sil = float(silhouette_score(embeddings, pred_labels, metric="euclidean"))
    ch  = float(calinski_harabasz_score(embeddings, pred_labels))
    db  = float(davies_bouldin_score(embeddings, pred_labels))
    return {"silhouette": sil, "calinski_harabasz": ch, "davies_bouldin": db}

quality_pub = evaluate_unsupervised_quality(Z_pub, pred_pub)
print("라벨 없이 보는 품질 지표 (Public‑Val):", quality_pub)


라벨 없이 보는 품질 지표 (Public‑Val): {'silhouette': 0.178085058927536, 'calinski_harabasz': 9.1228609085083, 'davies_bouldin': 1.1605395080330183}



## 8) 제출 리포트 템플릿 (텍스트 셀로 작성)

아래 항목을 그대로 복사해 작성 후, 결과 수치/분석을 채워 제출하세요.

**문제 요약**  
- AR(라벨 없음)로 비지도 표현 학습 → Yale(비식별화) 예측 생성  
- Public‑Val: 성능 확인(Hungarian ACC / NMI / ARI), Private‑Test: 최종 채점

**데이터/전처리**  
- AR: `data_ar/ar_database_stacked_128x128.npz` (float32, 0~1 정규화, 128×128 흑백)  
- Yale: `dist_students/all/`, `public_val_index.csv`, `private_test_index.csv`  
- 공통 전처리: 흑백, 128×128, 표준화/정규화\
(**그 밖에 어떤 전처리를 했는지 설명하세요.**)
    - (1) HOG: 조명에 강건한 윤곽선 및 기울기 정보 추출
    - (2) LBP: 얼굴 피부의 미세한 질감 패턴 추출
    - (3) Gamma Correction + Raw Pixel: 감마 보정으로 그림자를 완화한 후 픽셀 정보를 결합하여 전반적인 톤 정보를 보완
    - (4) StandardScaler로 스케일을 맞춘 후, L2 Normalization을 적용하여 벡터의 밝기가 아닌 유사도를 기준으로 군집화되도록 유도

**모델/학습(나의 구성)**:
**(모델을 어떻게 구성했는지 설명하세요.)**\
- 얼굴 특징 만들기:먼저 얼굴 이미지에 감마 보정, HOG, LBP, 그리고 축소된  픽셀 정보를 결합해 조명 변화와 표정 차이에 덜 민감한 특징 벡터를 만듦
- PCA로 차원 축소: 만든 특징 벡터가 너무 고차원이어서 AR 데이터로 PCA를 학습한 뒤 Yale 데이터는 transform만 해서 중요한 정보만 남기도록 차원을 축소\
- L2 정규화: Spectral Clustering이 잘 작동하도록 모든 벡터의 길이를 1로 맞추어 방향 정보만 비교하도록 정규화\
- Spectral Clustering 적용: 단순 K-Means보다 얼굴 데이터 구조를 더 잘 반영하기 위해 Nearest Neighbors 기반 Spectral Clustering을 사용, PCA 비율과 이웃 수를 여러 조합으로 시험해보며 Public-Val ACC가 가장 높은 조합을 선택

**Public‑Val 평가 결과 (숫자로 기입)**  
- Hungarian ACC: 0.755556  
- NMI: 0.858746
- ARI: 0.590955

**(선택) 라벨 없는 지표**
- silhouette: 0.178085058927536  
- calinski_harabasz: 9.1228609085083
- davies_bouldin: 1.1605395080330183

**분석/개선 계획**  
- 관찰/오류 사례, 개선 아이디어(SimCLR/BYOL/AutoEncoder, 증강/차원/K 설정 등)
  - 처음에 단순히 K-Means를 사용했을 때는 조명 차이 때문에 얼굴을 잘 구분하지 못해 약 37% 정도였음
  - 이후 HOG + LBP 특징을 사용하면서 조명 영향을 덜 받게 되어 성능이 약간 올라감
  - 마지막으로 스펙트럼 클러스터링을 적용해 단순 거리 기반이 아닌 데이터 간 연결 구조를 활용하면서 ARI가 0.19 → 0.59까지 개선됨
  - 이 이후 무슨 짓을 해도 올라가지 않아서 종료



## 9) 채점 결과 (제출한 "submission.csv" 파일을 기반으로 평가 예정)


